# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [28]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import euclidean, pdist, squareform

In [89]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [30]:
print(data.shape)
data.head()

(68584, 7)


,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [31]:
data2 = data.groupby(['CustomerID','ProductName']).sum()
print(data2.shape)
data2.head(3)

(63628, 3)


SalesID  ProductID  Quantity
CustomerID ProductName                                          
33         Apricots - Dried          172592        324         1
           Assorted Desserts        5363705        219         1
           Bandage - Flexible Neon  6299165        273         1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [32]:
pivot = pd.pivot_table(data, values='Quantity', index=['ProductName'], 
                       columns=['CustomerID'], aggfunc=np.sum).fillna(0)
pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [43]:
cussim = pd.DataFrame(squareform(pdist(pivot.T, 'euclidean')), 
                         index=pivot.columns, columns=pivot.columns)
cussim.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.000000,11.916375,10.488088,11.224972,11.401754,11.090537,12.409674,11.045361,11.269428,11.489125,...,206.871941,213.180675,225.656819,198.232187,230.913404,220.501701,217.188858,228.628520,239.000000,229.773802
200,11.916375,0.000000,11.747340,12.083046,12.569805,12.288206,12.165525,12.083046,11.874342,12.000000,...,206.310446,212.635839,224.697575,197.139544,230.952376,220.202180,215.728997,228.010965,239.037654,229.704158
264,10.488088,11.747340,0.000000,11.489125,11.224972,11.445523,12.000000,11.401754,11.180340,11.747340,...,206.387984,212.946003,225.435135,197.600607,230.371439,219.136943,216.612557,228.081126,238.266657,229.773802
356,11.224972,12.083046,11.489125,0.000000,12.083046,11.789826,12.328828,11.135529,11.958261,12.165525,...,206.649462,213.082144,225.452878,197.494304,231.038958,219.952268,217.437347,228.098663,238.493186,229.464594
412,11.401754,12.569805,11.224972,12.083046,0.000000,11.704700,12.328828,11.135529,11.789826,11.747340,...,206.900942,211.679002,225.572605,197.630969,230.614397,219.733930,217.446545,227.997807,238.396728,228.927936


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [59]:
topcus_200 = cussim[200].sort_values()[1:6]
print([e for e in topcus_200.index])

[3317, 1072, 3535, 1920, 3909]


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [61]:
top_5 = data2.loc[[e for e in topcus_200.index]]
top_5.head()

SalesID  ProductID  Quantity
CustomerID ProductName                                                    
1072       Appetizer - Mini Egg Roll, Shrimp  4770059        327         1
           Appetizer - Sausage Rolls          6604526         73         1
           Beans - Kidney, Canned             1148109        300         1
           Beef - Montreal Smoked Brisket      705042        373         1
           Beef - Striploin Aa                1804237        339         1

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [75]:
product = top_5.groupby(['ProductName']).sum().sort_values(by=['Quantity'],ascending=False)
product.head()

,SalesID,ProductID,Quantity
ProductName,,,
Soup - Campbells Bean Medley,15576869,1528,4
Muffin - Carrot Individual Wrap,13370132,1110,3
Bay Leaf,9117117,606,3
Pork - Kidney,13668367,150,3
"Pepper - Black, Whole",7058175,561,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [96]:
product_list = set(data.ProductName.values.tolist())
buy = pivot.index[pivot[200] > 0].tolist()
not_buy = pivot.index[pivot[200] == 0].tolist()

recom = []
for e in product.index:
    if len(recom)==5:
        break
    else:
        if e not in buy:
            recom.append(e)
print(recom)

['Soup - Campbells Bean Medley', 'Muffin - Carrot Individual Wrap', 'Bay Leaf', 'Pork - Kidney', 'Wanton Wrap']


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [114]:
pivot = pd.pivot_table(data, values='Quantity', index=['ProductName'], 
                       columns=['CustomerID'], aggfunc=np.sum).fillna(0)

cussim = pd.DataFrame(squareform(pdist(pivot.T, 'euclidean')), 
                      index=pivot.columns, columns=pivot.columns)

def prod_list(customerID):
    topcus = cussim[customerID].sort_values()[1:6]
    top_5 = data2.loc[[e for e in topcus.index]]
    product_index = top_5.groupby(['ProductName']).sum().sort_values(by=['Quantity'],ascending=False).index
    buy = pivot.index[pivot[customerID] > 0].tolist()
    recom = []
    for e in product_index:
        if len(recom)==5:
            break
        else:
            if e not in buy:
                recom.append(e)
    return recom

prod_list(3535)

['Butter - Unsalted',
 'Muffin - Carrot Individual Wrap',
 'Otomegusa Dashi Konbu',
 'Knife Plastic - White',
 'Lemonade - Natural, 591 Ml']

In [118]:
uniq_cus = set(data.CustomerID.values.tolist())
len(uniq_cus)

dicc = {}

for e in uniq_cus:
    dicc.update({f'{e}': prod_list(e)})

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [123]:
recomend = pd.DataFrame.from_dict(dicc)
recomendations = recomend.T
recomendations.columns = ['product_1', 'product_2','product_3','product_4','product_5']
recomendations.head()

,product_1,product_2,product_3,product_4,product_5
83973,Cocoa Butter,Veal - Slab Bacon,Bandage - Fexible 1x3,Ezy Change Mophandle,Chicken - Wieners
59399,"Shrimp - Baby, Warm Water",Knife Plastic - White,Towels - Paper / Kraft,Nantucket - Pomegranate Pear,Snapple - Iced Tea Peach
92168,Cup - Translucent 7 Oz Clear,"Veal - Brisket, Provimi,bnls",Snapple Lemon Tea,Spinach - Baby,Pop Shoppe Cream Soda
49159,"Pasta - Penne, Rigate, Dry",Cookies - Assorted,Puree - Mocha,Cheese - Parmesan Grated,Cheese - Victor Et Berthold
18441,Soupfoamcont12oz 112con,Anchovy Paste - 56 G Tube,Sprouts - Baby Pea Tendrils,"Lemonade - Natural, 591 Ml",Sausage - Breakfast


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [125]:
pivot = pd.pivot_table(data, values='Quantity', index=['ProductName'], 
                       columns=['CustomerID'], aggfunc=np.sum).fillna(0)

cussimne = pd.DataFrame(squareform(pdist(pivot.T, 'hamming')), 
                      index=pivot.columns, columns=pivot.columns)

def prod_list_ne(customerID):
    topcus = cussimne[customerID].sort_values()[1:6]
    top_5 = data2.loc[[e for e in topcus.index]]
    product_index = top_5.groupby(['ProductName']).sum().sort_values(by=['Quantity'],ascending=False).index
    buy = pivot.index[pivot[customerID] > 0].tolist()
    recom = []
    for e in product_index:
        if len(recom)==5:
            break
        else:
            if e not in buy:
                recom.append(e)
    return recom

prod_list(3535)

['Butter - Unsalted',
 'Muffin - Carrot Individual Wrap',
 'Otomegusa Dashi Konbu',
 'Knife Plastic - White',
 'Lemonade - Natural, 591 Ml']

In [126]:
uniq_cus = set(data.CustomerID.values.tolist())
len(uniq_cus)

diccne = {}

for e in uniq_cus:
    diccne.update({f'{e}': prod_list_ne(e)})

In [128]:
recomne = pd.DataFrame.from_dict(diccne)
recomendne = recomne.T
recomendne.columns = ['product_1', 'product_2','product_3','product_4','product_5']
display(recomendne.head())
display(recomendations.head())

,product_1,product_2,product_3,product_4,product_5
83973,"Lamb - Pieces, Diced",Garbag Bags - Black,"Fish - Scallops, Cold Smoked",Cake - Mini Cheesecake,Crab - Imitation Flakes
59399,"Oil - Shortening,liqud, Fry","Peas - Pigeon, Dry","Doilies - 5, Paper",Cheese - Parmesan Cubes,Pail For Lid 1537
92168,"Appetizer - Mini Egg Roll, Shrimp","Crab - Dungeness, Whole",Wiberg Super Cure,Cake - Mini Cheesecake,Wine - Prosecco Valdobiaddene
49159,"Lamb - Pieces, Diced","Mushrooms - Black, Dried",Mussels - Frozen,Rambutan,Beef Wellington
18441,"Mushrooms - Black, Dried",Mussels - Frozen,Kiwi,"Wine - Cahors Ac 2000, Clos",Rambutan


,product_1,product_2,product_3,product_4,product_5
83973,Cocoa Butter,Veal - Slab Bacon,Bandage - Fexible 1x3,Ezy Change Mophandle,Chicken - Wieners
59399,"Shrimp - Baby, Warm Water",Knife Plastic - White,Towels - Paper / Kraft,Nantucket - Pomegranate Pear,Snapple - Iced Tea Peach
92168,Cup - Translucent 7 Oz Clear,"Veal - Brisket, Provimi,bnls",Snapple Lemon Tea,Spinach - Baby,Pop Shoppe Cream Soda
49159,"Pasta - Penne, Rigate, Dry",Cookies - Assorted,Puree - Mocha,Cheese - Parmesan Grated,Cheese - Victor Et Berthold
18441,Soupfoamcont12oz 112con,Anchovy Paste - 56 G Tube,Sprouts - Baby Pea Tendrils,"Lemonade - Natural, 591 Ml",Sausage - Breakfast
